# Question 1

In [1]:
# symbolic computation tools
import sympy as sp
from sympy import symbols, pprint
from sympy import sin, cos, asin, acos, pi, lambdify
from sympy import Matrix, simplify, Function, diff, Derivative, nsimplify

from scipy.integrate import solve_ivp

import numpy as np

from IPython import display # for the animation
import matplotlib as mpl
mpl.use('Qt5Agg')

import matplotlib.pyplot as plt
plt.ion()


# styling for plots
mpl.rcParams['axes.titlesize'] = 24
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
    


## Question 1.1

In [13]:
# We wrap in parentheses here so we can write it on multiple lines. Similar
# with the triple quotes on the string. Usually we don't need to use these things.
(t, 
 theta_1, 
 theta_2, 
 theta_3, 
 l_1, 
 l_2, 
 l_3, 
 m_1,
 m_2,
 m_3,
 g) = symbols("""t, 
                         theta_1 
                         theta_2 
                         theta_3 
                         l_1 
                         l_2 
                         l_3
                         m_1
                         m_2
                         m_3
                         g""" , real = True)


theta_1 = Function('theta_1', real=True)(t)
theta_2 = Function('theta_2', real=True)(t)
theta_3 = Function('theta_3', real=True)(t)

In [3]:
def T(theta, x, y):
    """
    Function to return an arbitrary transformation matrix 
    This is for sympy symbolic calculation
    """
    return Matrix([[cos(theta), -sin(theta), x], 
                   [sin(theta), cos(theta), y],
                   [0, 0, 1]])

def sym_to_np(T):
    return np.array(T).astype(np.float64)

In [5]:
T_01 = T(theta_1, 0, 0)
T_12 = T(theta_2, l_1, 0)
T_23 = T(theta_3, l_2, 0)


In [6]:
FK1 = (T_01 * Matrix([[l_1], [0], [1]]))[:-1,0]
FK1

Matrix([
[l_1*cos(theta_1(t))],
[l_1*sin(theta_1(t))]])

In [7]:
FK2 = simplify((T_01 * T_12 * Matrix([[l_2], [0], [1]]))[:-1,0])
FK2

Matrix([
[l_1*cos(theta_1(t)) + l_2*cos(theta_1(t) + theta_2(t))],
[l_1*sin(theta_1(t)) + l_2*sin(theta_1(t) + theta_2(t))]])

In [11]:
FK3 = simplify((T_01 * T_12 * T_23 * Matrix([[l_3], [0], [1]]))[:-1,0])
FK3

Matrix([
[l_1*cos(theta_1(t)) + l_2*cos(theta_1(t) + theta_2(t)) + l_3*cos(theta_1(t) + theta_2(t) + theta_3(t))],
[l_1*sin(theta_1(t)) + l_2*sin(theta_1(t) + theta_2(t)) + l_3*sin(theta_1(t) + theta_2(t) + theta_3(t))]])

In [8]:
v1 = diff(FK1,t)
KE1 = (1/2)*m_1*(v1.T * v1)[0]
KE1

0.5*m_1*(l_1**2*sin(theta_1(t))**2*Derivative(theta_1(t), t)**2 + l_1**2*cos(theta_1(t))**2*Derivative(theta_1(t), t)**2)

In [9]:
v2 = diff(FK2,t)
KE2 = (1/2)*m_2*(v2.T * v2)[0]
KE2

0.5*m_2*((-l_1*sin(theta_1(t))*Derivative(theta_1(t), t) - l_2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*sin(theta_1(t) + theta_2(t)))**2 + (l_1*cos(theta_1(t))*Derivative(theta_1(t), t) + l_2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*cos(theta_1(t) + theta_2(t)))**2)

In [14]:
v3 = diff(FK3,t)
KE3 = (1/2)*m_3*(v3.T * v3)[0]
KE3

0.5*m_3*((-l_1*sin(theta_1(t))*Derivative(theta_1(t), t) - l_2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*sin(theta_1(t) + theta_2(t)) - l_3*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t) + Derivative(theta_3(t), t))*sin(theta_1(t) + theta_2(t) + theta_3(t)))**2 + (l_1*cos(theta_1(t))*Derivative(theta_1(t), t) + l_2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*cos(theta_1(t) + theta_2(t)) + l_3*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t) + Derivative(theta_3(t), t))*cos(theta_1(t) + theta_2(t) + theta_3(t)))**2)

In [18]:
T = KE1 + KE2 + KE3
V = m_1*g*FK1[1] + m_2*g*FK2[1] + m_3*g*FK3[1]

L = T - V
L = simplify(L)
L

-g*l_1*m_1*sin(theta_1(t)) - g*m_2*(l_1*sin(theta_1(t)) + l_2*sin(theta_1(t) + theta_2(t))) - g*m_3*(l_1*sin(theta_1(t)) + l_2*sin(theta_1(t) + theta_2(t)) + l_3*sin(theta_1(t) + theta_2(t) + theta_3(t))) + 0.5*l_1**2*m_1*Derivative(theta_1(t), t)**2 + 0.5*m_2*(l_1**2*Derivative(theta_1(t), t)**2 + 2*l_1*l_2*cos(theta_2(t))*Derivative(theta_1(t), t)**2 + 2*l_1*l_2*cos(theta_2(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) + l_2**2*Derivative(theta_1(t), t)**2 + 2*l_2**2*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) + l_2**2*Derivative(theta_2(t), t)**2) + 0.5*m_3*((l_1*sin(theta_1(t))*Derivative(theta_1(t), t) + l_2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*sin(theta_1(t) + theta_2(t)) + l_3*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t) + Derivative(theta_3(t), t))*sin(theta_1(t) + theta_2(t) + theta_3(t)))**2 + (l_1*cos(theta_1(t))*Derivative(theta_1(t), t) + l_2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*cos(theta_1(t) + theta_2(t)) 

In [19]:
EOM_theta_1 = diff(diff(L, Derivative(theta_1, t)), t) - diff(L, theta_1)
EOM_theta_1 = nsimplify(EOM_theta_1)
EOM_theta_1

g*l_1*m_1*cos(theta_1(t)) + g*m_2*(l_1*cos(theta_1(t)) + l_2*cos(theta_1(t) + theta_2(t))) + g*m_3*(l_1*cos(theta_1(t)) + l_2*cos(theta_1(t) + theta_2(t)) + l_3*cos(theta_1(t) + theta_2(t) + theta_3(t))) + l_1**2*m_1*Derivative(theta_1(t), (t, 2)) + m_2*(2*l_1**2*Derivative(theta_1(t), (t, 2)) - 4*l_1*l_2*sin(theta_2(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) - 2*l_1*l_2*sin(theta_2(t))*Derivative(theta_2(t), t)**2 + 4*l_1*l_2*cos(theta_2(t))*Derivative(theta_1(t), (t, 2)) + 2*l_1*l_2*cos(theta_2(t))*Derivative(theta_2(t), (t, 2)) + 2*l_2**2*Derivative(theta_1(t), (t, 2)) + 2*l_2**2*Derivative(theta_2(t), (t, 2)))/2 - m_3*((-2*l_1*sin(theta_1(t))*Derivative(theta_1(t), t) - 2*l_2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*sin(theta_1(t) + theta_2(t)) - 2*l_3*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t) + Derivative(theta_3(t), t))*sin(theta_1(t) + theta_2(t) + theta_3(t)))*(l_1*cos(theta_1(t))*Derivative(theta_1(t), t) + l_2*(Derivative(theta_1(t), t

In [20]:
EOM_theta_2 = diff(diff(L, Derivative(theta_2, t)), t) - diff(L, theta_2)
EOM_theta_2 = nsimplify(EOM_theta_2)
EOM_theta_2

g*l_2*m_2*cos(theta_1(t) + theta_2(t)) + g*m_3*(l_2*cos(theta_1(t) + theta_2(t)) + l_3*cos(theta_1(t) + theta_2(t) + theta_3(t))) - m_2*(-2*l_1*l_2*sin(theta_2(t))*Derivative(theta_1(t), t)**2 - 2*l_1*l_2*sin(theta_2(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t))/2 + m_2*(-2*l_1*l_2*sin(theta_2(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) + 2*l_1*l_2*cos(theta_2(t))*Derivative(theta_1(t), (t, 2)) + 2*l_2**2*Derivative(theta_1(t), (t, 2)) + 2*l_2**2*Derivative(theta_2(t), (t, 2)))/2 - m_3*((-2*l_2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*sin(theta_1(t) + theta_2(t)) - 2*l_3*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t) + Derivative(theta_3(t), t))*sin(theta_1(t) + theta_2(t) + theta_3(t)))*(l_1*cos(theta_1(t))*Derivative(theta_1(t), t) + l_2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*cos(theta_1(t) + theta_2(t)) + l_3*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t) + Derivative(theta_3(t), t))*cos(theta_1(t) + theta_2(t) +

In [21]:
EOM_theta_3 = diff(diff(L, Derivative(theta_3, t)), t) - diff(L, theta_3)
EOM_theta_3 = nsimplify(EOM_theta_3)
EOM_theta_3

g*l_3*m_3*cos(theta_1(t) + theta_2(t) + theta_3(t)) - m_3*(2*l_3*(l_1*sin(theta_1(t))*Derivative(theta_1(t), t) + l_2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*sin(theta_1(t) + theta_2(t)) + l_3*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t) + Derivative(theta_3(t), t))*sin(theta_1(t) + theta_2(t) + theta_3(t)))*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t) + Derivative(theta_3(t), t))*cos(theta_1(t) + theta_2(t) + theta_3(t)) - 2*l_3*(l_1*cos(theta_1(t))*Derivative(theta_1(t), t) + l_2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*cos(theta_1(t) + theta_2(t)) + l_3*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t) + Derivative(theta_3(t), t))*cos(theta_1(t) + theta_2(t) + theta_3(t)))*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t) + Derivative(theta_3(t), t))*sin(theta_1(t) + theta_2(t) + theta_3(t)))/2 + m_3*(2*l_3*(l_1*sin(theta_1(t))*Derivative(theta_1(t), t) + l_2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*sin(thet

In [23]:
mass = sp.symarray('',(3,3))

mass[0,0] = EOM_theta_1.expand().coeff(Derivative(theta_1,t,t))

mass[0,1] = EOM_theta_1.expand().coeff(Derivative(theta_2,t,t))
mass[1,0] = mass[0,1]

mass[0,2] = EOM_theta_1.expand().coeff(Derivative(theta_3,t,t))
mass[2,0] = mass[0,2]

mass[1,1] = EOM_theta_2.expand().coeff(Derivative(theta_2,t,t))

mass[1,2] = EOM_theta_2.expand().coeff(Derivative(theta_3,t,t))
mass[2,1] = mass[1,2]

mass[2,2] = EOM_theta_3.expand().coeff(Derivative(theta_3,t,t))

mass = Matrix(mass)

In [25]:
mass = simplify(mass)
mass

Matrix([
[l_1**2*m_1 + l_1**2*m_2 + l_1**2*m_3 + 2*l_1*l_2*m_2*cos(theta_2(t)) + 2*l_1*l_2*m_3*cos(theta_2(t)) + 2*l_1*l_3*m_3*cos(theta_2(t) + theta_3(t)) + l_2**2*m_2 + l_2**2*m_3 + 2*l_2*l_3*m_3*cos(theta_3(t)) + l_3**2*m_3, l_1*l_2*m_2*cos(theta_2(t)) + l_1*l_2*m_3*cos(theta_2(t)) + l_1*l_3*m_3*cos(theta_2(t) + theta_3(t)) + l_2**2*m_2 + l_2**2*m_3 + 2*l_2*l_3*m_3*cos(theta_3(t)) + l_3**2*m_3, l_3*m_3*(l_1*cos(theta_2(t) + theta_3(t)) + l_2*cos(theta_3(t)) + l_3)],
[                                             l_1*l_2*m_2*cos(theta_2(t)) + l_1*l_2*m_3*cos(theta_2(t)) + l_1*l_3*m_3*cos(theta_2(t) + theta_3(t)) + l_2**2*m_2 + l_2**2*m_3 + 2*l_2*l_3*m_3*cos(theta_3(t)) + l_3**2*m_3,                                                                                                        l_2**2*m_2 + l_2**2*m_3 + 2*l_2*l_3*m_3*cos(theta_3(t)) + l_3**2*m_3,                                    l_3*m_3*(l_2*cos(theta_3(t)) + l_3)],
[                                                            

The mass matrix generated from Euler-Lagrange compared to the one from the previous homework is the same!

## Question 1.2

In [33]:
accel_state_vector = Matrix([[Derivative(theta_1, t, t)], [Derivative(theta_2, t, t)], [Derivative(theta_3, t, t)]])

EOM1_leftover = simplify(EOM_theta_1.expand() - (mass[0,:]*accel_state_vector)[0])
EOM2_leftover = simplify(EOM_theta_2.expand() - (mass[1,:]*accel_state_vector)[0])
EOM3_leftover = simplify(EOM_theta_3.expand() - (mass[2,:]*accel_state_vector)[0])

In [35]:
EOM1_leftover

g*l_1*m_1*cos(theta_1(t)) + g*l_1*m_2*cos(theta_1(t)) + g*l_1*m_3*cos(theta_1(t)) + g*l_2*m_2*cos(theta_1(t) + theta_2(t)) + g*l_2*m_3*cos(theta_1(t) + theta_2(t)) + g*l_3*m_3*cos(theta_1(t) + theta_2(t) + theta_3(t)) - 2*l_1*l_2*m_2*sin(theta_2(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) - l_1*l_2*m_2*sin(theta_2(t))*Derivative(theta_2(t), t)**2 - 2*l_1*l_2*m_3*sin(theta_2(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) - l_1*l_2*m_3*sin(theta_2(t))*Derivative(theta_2(t), t)**2 - 2*l_1*l_3*m_3*sin(theta_2(t) + theta_3(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) - 2*l_1*l_3*m_3*sin(theta_2(t) + theta_3(t))*Derivative(theta_1(t), t)*Derivative(theta_3(t), t) - l_1*l_3*m_3*sin(theta_2(t) + theta_3(t))*Derivative(theta_2(t), t)**2 - 2*l_1*l_3*m_3*sin(theta_2(t) + theta_3(t))*Derivative(theta_2(t), t)*Derivative(theta_3(t), t) - l_1*l_3*m_3*sin(theta_2(t) + theta_3(t))*Derivative(theta_3(t), t)**2 - 2*l_2*l_3*m_3*sin(theta_3(t))*Derivative(theta_1(t), t)*Der

In [36]:
EOM2_leftover

g*l_2*m_2*cos(theta_1(t) + theta_2(t)) + g*l_2*m_3*cos(theta_1(t) + theta_2(t)) + g*l_3*m_3*cos(theta_1(t) + theta_2(t) + theta_3(t)) + l_1*l_2*m_2*sin(theta_2(t))*Derivative(theta_1(t), t)**2 + l_1*l_2*m_3*sin(theta_2(t))*Derivative(theta_1(t), t)**2 + l_1*l_3*m_3*sin(theta_2(t) + theta_3(t))*Derivative(theta_1(t), t)**2 - 2*l_2*l_3*m_3*sin(theta_3(t))*Derivative(theta_1(t), t)*Derivative(theta_3(t), t) - 2*l_2*l_3*m_3*sin(theta_3(t))*Derivative(theta_2(t), t)*Derivative(theta_3(t), t) - l_2*l_3*m_3*sin(theta_3(t))*Derivative(theta_3(t), t)**2

In [37]:
EOM3_leftover

l_3*m_3*(g*cos(theta_1(t) + theta_2(t) + theta_3(t)) + l_1*sin(theta_2(t) + theta_3(t))*Derivative(theta_1(t), t)**2 + l_2*sin(theta_3(t))*Derivative(theta_1(t), t)**2 + 2*l_2*sin(theta_3(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) + l_2*sin(theta_3(t))*Derivative(theta_2(t), t)**2)

In [39]:
G_1 = EOM1_leftover.subs([(Derivative(theta_1, t), 0), (Derivative(theta_2, t), 0), (Derivative(theta_3, t), 0)])
G_1

g*l_1*m_1*cos(theta_1(t)) + g*l_1*m_2*cos(theta_1(t)) + g*l_1*m_3*cos(theta_1(t)) + g*l_2*m_2*cos(theta_1(t) + theta_2(t)) + g*l_2*m_3*cos(theta_1(t) + theta_2(t)) + g*l_3*m_3*cos(theta_1(t) + theta_2(t) + theta_3(t))

In [40]:
G_2 = EOM2_leftover.subs([(Derivative(theta_1, t), 0), (Derivative(theta_2, t), 0), (Derivative(theta_3, t), 0)])
G_2

g*l_2*m_2*cos(theta_1(t) + theta_2(t)) + g*l_2*m_3*cos(theta_1(t) + theta_2(t)) + g*l_3*m_3*cos(theta_1(t) + theta_2(t) + theta_3(t))

In [42]:
G_3 = EOM3_leftover.subs([(Derivative(theta_1, t), 0), (Derivative(theta_2, t), 0), (Derivative(theta_3, t), 0)])
G_3

g*l_3*m_3*cos(theta_1(t) + theta_2(t) + theta_3(t))

In [46]:
C_1 = EOM1_leftover - G_1
C_1

-2*l_1*l_2*m_2*sin(theta_2(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) - l_1*l_2*m_2*sin(theta_2(t))*Derivative(theta_2(t), t)**2 - 2*l_1*l_2*m_3*sin(theta_2(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) - l_1*l_2*m_3*sin(theta_2(t))*Derivative(theta_2(t), t)**2 - 2*l_1*l_3*m_3*sin(theta_2(t) + theta_3(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) - 2*l_1*l_3*m_3*sin(theta_2(t) + theta_3(t))*Derivative(theta_1(t), t)*Derivative(theta_3(t), t) - l_1*l_3*m_3*sin(theta_2(t) + theta_3(t))*Derivative(theta_2(t), t)**2 - 2*l_1*l_3*m_3*sin(theta_2(t) + theta_3(t))*Derivative(theta_2(t), t)*Derivative(theta_3(t), t) - l_1*l_3*m_3*sin(theta_2(t) + theta_3(t))*Derivative(theta_3(t), t)**2 - 2*l_2*l_3*m_3*sin(theta_3(t))*Derivative(theta_1(t), t)*Derivative(theta_3(t), t) - 2*l_2*l_3*m_3*sin(theta_3(t))*Derivative(theta_2(t), t)*Derivative(theta_3(t), t) - l_2*l_3*m_3*sin(theta_3(t))*Derivative(theta_3(t), t)**2

In [44]:
C_2 = simplify(EOM2_leftover - G_2)
C_2

l_1*l_2*m_2*sin(theta_2(t))*Derivative(theta_1(t), t)**2 + l_1*l_2*m_3*sin(theta_2(t))*Derivative(theta_1(t), t)**2 + l_1*l_3*m_3*sin(theta_2(t) + theta_3(t))*Derivative(theta_1(t), t)**2 - 2*l_2*l_3*m_3*sin(theta_3(t))*Derivative(theta_1(t), t)*Derivative(theta_3(t), t) - 2*l_2*l_3*m_3*sin(theta_3(t))*Derivative(theta_2(t), t)*Derivative(theta_3(t), t) - l_2*l_3*m_3*sin(theta_3(t))*Derivative(theta_3(t), t)**2

In [47]:
C_3 = simplify(EOM3_leftover - G_3)
C_3

l_3*m_3*(l_1*sin(theta_2(t) + theta_3(t))*Derivative(theta_1(t), t)**2 + l_2*sin(theta_3(t))*Derivative(theta_1(t), t)**2 + 2*l_2*sin(theta_3(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) + l_2*sin(theta_3(t))*Derivative(theta_2(t), t)**2)

In [ ]:
simplify(Matrix([[EOM_theta_1], [EOM_theta_2], [EOM_theta_3]]) - (mass*accel_state_vector + Matrix([[C_1], [C_2], [C_3]]) + Matrix([[G_1], [G_2], [G_3]])))